<a href="https://colab.research.google.com/github/jserra22/coderhouse/blob/main/coder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-generativeai python-pptx python-docx PyPDF2 pytesseract Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.8/472.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 10.1 MB/s eta 0:00:00


In [ ]:
import os
import google.generativeai as genai
from google.colab import userdata, files
import PyPDF2
import zlib
import docx
import pptx
from PIL import Image
import pytesseract
from google.genai import types
from io import BytesIO
from google import genai as gemini_genai
import time
from pptx import Presentation
from pptx.util import Inches

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

def extraer_texto_pdf(nombre_archivo):
    """Extrae texto de un archivo PDF, manejando la descompresión."""
    try:
        with open(nombre_archivo, 'rb') as archivo_pdf:
            lector_pdf = PyPDF2.PdfReader(archivo_pdf)
            texto_completo = ""
            for pagina in lector_pdf.pages:
                contenido_pagina = pagina.extract_text()
                if contenido_pagina:
                    texto_completo += contenido_pagina
                else:
                    try:
                        contenido_crudo = pagina.extract_text(None, True)
                        if contenido_crudo:
                            texto_completo += zlib.decompress(contenido_crudo).decode('utf-8')
                    except:
                        pass
        return texto_completo
    except FileNotFoundError:
        return "Archivo no encontrado."
    except Exception as e:
        return f"Ocurrió un error: {e}"

def extraer_texto_docx(nombre_archivo):
    """Extrae texto de un archivo DOCX."""
    try:
        doc = docx.Document(nombre_archivo)
        texto_completo = ""
        for párrafo in doc.paragraphs:
            texto_completo += párrafo.text + "\n"
        return texto_completo
    except Exception as e:
        return f"Ocurrió un error al procesar DOCX: {e}"

def extraer_texto_doc(nombre_archivo):
    """Extrae texto de un archivo DOC."""
    try:
        return "No se soporta la extracción directa de archivos .doc. Por favor, convierta el archivo a .docx."
    except:
        return "No se soporta la extracción directa de archivos .doc. Por favor, convierta el archivo a .docx."

def extraer_texto_pptx(nombre_archivo):
    """Extrae texto de un archivo PPTX."""
    try:
        ppt = pptx.Presentation(nombre_archivo)
        texto_completo = ""
        for slide in ppt.slides:
            for shape in slide.shapes:
                if shape.has_text_frame:
                    for paragraph in shape.text_frame.paragraphs:
                        for run in paragraph.runs:
                            texto_completo += run.text
                    texto_completo += "\n"
        return texto_completo
    except Exception as e:
        return f"Ocurrió un error al procesar PPTX: {e}"

def extraer_texto_png(nombre_archivo):
    """Extrae texto de un archivo PNG usando OCR."""
    try:
        img = Image.open(nombre_archivo)
        texto = pytesseract.image_to_string(img)
        return texto
    except Exception as e:
        return f"Ocurrió un error al procesar PNG (OCR): {e}"

def generar_resumen(texto):
    """Genera un resumen del texto y extrae puntos clave."""
    prompt_resumen = """Eres un estudiante universitario.
Hacer un resumen del siguiente texto marcando las principales temáticas, personajes y categoria en la cual se puede encasillar al texto.
El resumen debe estar en un formato de bullet points para su facil exportación a Google Slides o PowerPoint.

Texto:
""" + texto

    model = genai.GenerativeModel('gemini-2.0-flash')
    response = model.generate_content(prompt_resumen)
    resumen = response.text

    puntos_clave = [linea.strip().replace("*", "").strip() for linea in resumen.split("\n") if linea.strip().startswith("*")]

    return resumen, puntos_clave

def generar_imagenes(puntos_clave):
    """Genera imágenes basadas en los puntos clave del resumen."""
    client = gemini_genai.Client()
    image_paths = []

    for i, punto in enumerate(puntos_clave[:4]):
        prompt_imagen = f"Crea una imagen que represente visualmente el siguiente concepto: {punto}"
        response = client.models.generate_content(
            model="gemini-2.0-flash-exp-image-generation",
            contents=prompt_imagen,
            config=types.GenerateContentConfig(
                response_modalities=['Text', 'Image']
            )
        )

        for part in response.candidates[0].content.parts:
            if part.inline_data is not None:
                image = Image.open(BytesIO((part.inline_data.data)))
                image_name = f'gemini-image-{int(time.time())}-{i}.png'
                image.save(image_name)
                image_paths.append(image_name)
                image.show()

    return image_paths

uploaded = files.upload()

if uploaded:
    nombre_archivo = list(uploaded.keys())[0]
    _, extension = os.path.splitext(nombre_archivo)
    extension = extension.lower()

    archivos_temporales = [nombre_archivo]

    if extension == ".pdf":
        texto = extraer_texto_pdf(nombre_archivo)
    elif extension == ".docx":
        texto = extraer_texto_docx(nombre_archivo)
    elif extension == ".doc":
        texto = extraer_texto_doc(nombre_archivo)
    elif extension == ".pptx":
        texto = extraer_texto_pptx(nombre_archivo)
    elif extension == ".png":
        texto = extraer_texto_png(nombre_archivo)
    else:
        texto = "Formato de archivo no soportado."

    if "Ocurrió un error" in texto or "Formato de archivo no soportado." in texto:
        print(texto)
    else:
        resumen, puntos_clave = generar_resumen(texto)
        print("Resumen:\n", resumen)
        print("\nPuntos clave:\n", puntos_clave)

        image_paths = generar_imagenes(puntos_clave)

        prs = Presentation()

        # Agregar las diapositivas de texto (resumen)
        slide_resumen = prs.slides.add_slide(prs.slide_layouts[1])  # layout con título y contenido
        titulo = slide_resumen.shapes.title
        contenido = slide_resumen.shapes.placeholders[1]  # Placeholder para el contenido

        titulo.text = "Resumen del texto"
        contenido.text_frame.text = resumen

        # Dividir el resumen en 3 partes y agregar a las siguientes diapositivas
        partes_resumen = resumen.split('\n')
        partes_diapositivas = [partes_resumen[i:i + len(partes_resumen) // 3] for i in range(0, len(partes_resumen), len(partes_resumen) // 3)]

        for i, parte in enumerate(partes_diapositivas):
            if i < 3:  # Asegurarse de que no se creen más de 3 diapositivas de texto
                slide_texto = prs.slides.add_slide(prs.slide_layouts[1])
                titulo_texto = slide_texto.shapes.title
                contenido_texto = slide_texto.shapes.placeholders[1]
                titulo_texto.text = f"Resumen (Parte {i + 1})"
                contenido_texto.text_frame.text = '\n'.join(parte)

        # Agregar las diapositivas de imágenes
        for image_path in image_paths:
            slide_imagen = prs.slides.add_slide(prs.slide_layouts[5]) # 5 = blank layout
            left = top = Inches(1)
            pic = slide_imagen.shapes.add_picture(image_path, left, top, width=Inches(8))

        prs.save('presentacion_gemini.pptx')

        files.download('presentacion_gemini.pptx')

    for archivo in archivos_temporales:
        try:
            os.remove(archivo)
            print(f"Archivo temporal eliminado: {archivo}")
        except FileNotFoundError:
            print(f"Archivo no encontrado: {archivo}")
        except Exception as e:
            print(f"Error al eliminar el archivo {archivo}: {e}")

else:
    print("No se seleccionó ningún archivo.")

Saving 2.1.146_Maquiavelo_El Principe.pdf to 2.1.146_Maquiavelo_El Principe.pdf
Resumen:
 ¡Por supuesto! Aquí tienes un resumen del texto "El Príncipe" en formato bullet points, ideal para presentaciones:

**Categoría:**

*   Tratado de doctrina política/Manual para gobernantes

**Temáticas Principales:**

*   **Adquisición y Mantenimiento del Poder:**
    *   Tipos de principados: hereditarios, nuevos, mixtos.
    *   Estrategias para gobernar cada tipo de principado.
    *   Importancia de la fortuna (azar) vs. la virtud (habilidad) del gobernante.
*   **El Arte de Gobernar:**
    *   Realismo político: enfocarse en la "verdad efectiva" en lugar de ideales utópicos.
    *   Crueldad y clemencia: cuándo y cómo usarlas.
    *   Ser temido vs. ser amado: la utilidad del temor para mantener el orden.
    *   La importancia de la apariencia: parecer virtuoso, aunque no siempre serlo.
*   **Fuerzas Militares y la Guerra:**
    *   Crítica a las milicias mercenarias y auxiliares.
    *   Én

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Archivo temporal eliminado: 2.1.146_Maquiavelo_El Principe.pdf
